In [ ]:
import os
import glob
import shutil
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from rapidfuzz import process, fuzz
import pandas as pd
import numpy as np
import re

#LASKUJEN KÄSITTELY

#projektilistaus 002 Urakat -kansiosta / VAIHTOEHTOISESTI PROJEKTILISTAUKSEN VOI TEHDÄ LISTANA
import os

project_folder_path = r"...\001 TOIMISTOURAKAT\002 URAKAT"

project_list = []

for i in os.listdir(project_folder_path):
    i_path = os.path.join(project_folder_path, i)
    if os.path.isdir(i_path):
        i = re.sub(r"^\d+\s?_?\d+?\s+?", "", i)
        if i not in ["MALLIKANSIORAKENNE", "MENNEET PIENET URAKAT", "Paraistentie"]:
            project_list.append(i)
            
#luodaan funktiot

#tällä funktiolla tsekataan kommentin urakka / lisätyö maininat. jos löytyy jompikumpi, niin palauttaa kaksi muuttujaa:
#tägin ja projektin. jos mainintaa ei ole, funktio palauttaa pelkän kommentin.

def check_additional(comment, keyword, threshold=60):
    comment = comment.lower().split()
    keyword = keyword.lower()
    for i, c in enumerate(comment):
        score = fuzz.ratio(c, keyword)
        if score >= threshold:
            project = " ".join(comment[:i])
            additional_tag = " ".join(comment[i:])
            return project, additional_tag

    project = " ".join(comment)
    return project, None

#tällä poimitaan projektinimet

def preprocess(comment):
    comment = comment.lower().strip()
    comment = re.sub(r"[^\w\s]", "", comment)
    comment = re.sub(r"\s+", " ", comment)
    return comment

def extract_project_name(comment, project_list, threshold = 70):
    comment = preprocess(comment)
    best_project_match = None
    best_score = 0
    
    for i, p in enumerate(project_list):
        p = preprocess(p)
        score = fuzz.partial_ratio(comment, p)
        
        if score > best_score:
            best_project_match = project_list[i]
            best_score = score

    if best_score >= threshold:

        
        return best_project_match

    else:
        return comment

#tuodaan kumulatiivinen ja ladatut laskut
data_cum = pd.read_excel(r"...\laskujen_selaus\kumulatiivinen_projektilaskujen_kirjanpito.xlsx")
data_cum = data_cum.dropna(subset=["Viimeisin kommentti"])
data_new = pd.read_excel(r"...\laskujen_selaus\laskut.xlsx")
data_new = data_new.dropna(subset=["Viimeisin kommentti"])
new_vs_cum_dif = len(data_new) - len(data_cum)

#pudotetaan tuplalaskut Toimittaja ja Laskunumero -sarakkeiden perusteella, muuten saattaa olla samoja numeroita historiassa
new_invoices = pd.concat([data_cum, data_new], ignore_index=True)
new_invoices["Laskunumero"] = new_invoices["Laskunumero"].apply(lambda x: re.sub(r"\D", "", str(x)))
new_invoices["Laskunumero"] = new_invoices["Laskunumero"].apply(lambda x: int(x.strip()))

mask = new_invoices.duplicated(subset=["Toimittaja", "Laskunumero"], keep=False)
new_invoices = new_invoices[~mask]
new_invoices_total = len(new_invoices)

#Kumulatiivisen ja uusien laskeminen yhteen ja vertailu käsiteltyyn listaan voi palauttaa väärän määrän laskuja. Oletettavasti sen takia,
#että lopussa palautetaan käsitellyt laskut (eli ei tyhjiä projekteille kuulumattomia mukana, toisin kun kumulatiivisessa)

# if new_vs_cum_dif - new_invoices_total != 0:
#     raise ValueError(f"Laskujen määrä ei täsmää, uusia laskuja {new_invoices_total}, kumulatiiviset vs uudet {new_vs_cum_dif}")

print(f"Laskut tarkastettu onnistuneesti, uusia laskuja: {new_invoices_total}")

#pudotetaan kumulatiiviseenkirjanpitoon meneviltä laskuilta tuplat
cum_data_cum_and_new = pd.concat([data_cum, data_new])
mask = cum_data_cum_and_new.duplicated(subset=["Toimittaja", "Laskunumero"], keep="first")
cum_data_cum_and_new = cum_data_cum_and_new[~mask]
cum_data_cum_and_new_total = len(cum_data_cum_and_new)

#tässä sama kuin edellä, kumulatiivisen vertailu voi olla hankala jos ei ole täysin varma vanhojen ja uusien laskujen käsittelyn
#samanlaisuudesta

# if cum_data_cum_and_new_total - len(data_new) != 0:
#     raise ValuError(f"Virhe kumulatiivisen ja uusien laskujen yhdistämisessä, erotus: {cum_data_cum_and_new_total - len(data_new)}")

# elif len(cum_data_cum_and_new) - len(data_cum) == 0:
#     raise ValueError("Ei uusia laskuja, tarkasta mikä vialla koodissa tai laskupohjissa")

print("Viedään uudet laskut kumulatiiviseen laskukirjanpitoon")
print(f"Uusia laskuja viedään kumulatiiviseen kirjanpitoon {cum_data_cum_and_new_total - len(data_cum)}")

cum_data_cum_and_new = cum_data_cum_and_new.sort_values("Laskun päiväys", ascending=False)
cum_data_cum_and_new.to_excel(r"...\laskujen_selaus\kumulatiivinen_projektilaskujen_kirjanpito.xlsx", index=False)

#UUSIEN LASKUJEN KÄSITTELY
print("Käsitellään uudet laskut")

#muutetaan päiväykset luettaviksi
new_invoices.loc[:,"Laskun päiväys"] = pd.to_datetime(new_invoices["Laskun päiväys"], format="%-d.%-m.-%Y")
new_invoices["Laskun päiväys"] = new_invoices["Laskun päiväys"].dt.strftime("%#d.%#m.%#Y")
print("Muutettu päiväykset luettaviksi")

#muokataan toimittaja-sarake luettavaksi
new_invoices["Toimittaja"] = new_invoices["Toimittaja"].apply(lambda x: re.sub(r"^\d*\s*|\s?\(.*\)", "", x.title()))
print("Muutettutoimittaja-sarake luettavaksi")

#lisätään toimittajien alvilistaus
alvit = pd.read_excel(r"...\laskujen_selaus\toimittajat.xlsx")
alvit = pd.DataFrame(alvit)
alvit["Toimittaja"] = alvit.loc[:,"Toimittaja"].apply(lambda x: re.sub(r"^\d*\s*|\s?\(.*\)", "", x.title()))
new_invoices = pd.merge(new_invoices, alvit, on="Toimittaja", how="left")
print("Lisätty toimittajien alv-listaus")

#lasketaan alvit pois summilta
new_invoices["Summa"] = np.where(new_invoices["alv"] == 1,
                                new_invoices["Summa"]/1.255,
                                new_invoices["Summa"])
print("Laskettu alvit pois summista")

#käydään uudet laskut läpi, poimitaan urakka / lisätyömaininat ja jaotellaan ne projektikohtaisesti

new_invoices[["Viimeisin kommentti", "Lisätyö"]] = new_invoices["Viimeisin kommentti"].apply(lambda comment: check_additional(comment, "lisätyö")).apply(pd.Series)
new_invoices[["Viimeisin kommentti", "Urakka"]] = new_invoices["Viimeisin kommentti"].apply(lambda comment: check_additional(comment, "urakka")).apply(pd.Series)

new_invoices["Projekti"] = new_invoices["Viimeisin kommentti"].apply(lambda comment: extract_project_name(comment, project_list))
new_invoices["Summa"] = new_invoices["Summa"].apply(lambda x: "{:.2f}".format(float(x)))
print("Poimittu lisätyöt ja urakat erikseen")

#poistetaan turhat sarakkeet ja lajitellaan projektien mukaisesti
new_invoices = new_invoices.drop(["Tositelaji", "Hyväksyjä", "Tila", "Eräpäivä", ], axis=1)
new_invoices = new_invoices.sort_values("Projekti")
print("Lajiteltu taulukko")

#viedään laskutettu kansioon
current_date = datetime.now().strftime("%d.%m.%Y")
filename = f"tarkistetut laskut {current_date}.xlsx"
new_invoices.to_excel(r"...\laskujen_selaus\laskulistat" + "\\" + filename, index=False)
print(f"Laskut viety kansioon nimellä {filename}")
print("Laskupohja valmis lajiteltavaksi")